In [1]:
import numpy as np
import pandas as pd



# Import danych

In [2]:
def read_data(path:str):
    data = pd.read_csv(train_file_path, sep="  ", skiprows=1, header=0)

    column_name = ["Class"]

    for index in range(1, len(data.columns)):
        column_name.append(f"P{index}")

    data.columns = column_name
    return data

In [3]:
train_file_path = "../data/iris_trn.txt"
test_file_path = "../data/iris_tst.txt"

train_data = read_data(train_file_path)
test_data = read_data(test_file_path)

train_data

/opt/conda/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,Class,P1,P2,P3,P4
0,1,4.9,3.0,1.4,0.2
1,1,4.7,3.2,1.3,0.2
2,1,4.6,3.1,1.5,0.2
3,1,5.0,3.6,1.4,0.2
4,1,5.4,3.9,1.7,0.4
...,...,...,...,...,...
69,3,6.9,3.2,5.7,2.3
70,3,5.6,2.8,4.9,2.0
71,3,7.7,2.8,6.7,2.0
72,3,6.3,2.7,4.9,1.8


In [4]:
test_data

,Class,P1,P2,P3,P4
0,1,4.9,3.0,1.4,0.2
1,1,4.7,3.2,1.3,0.2
2,1,4.6,3.1,1.5,0.2
3,1,5.0,3.6,1.4,0.2
4,1,5.4,3.9,1.7,0.4
...,...,...,...,...,...
69,3,6.9,3.2,5.7,2.3
70,3,5.6,2.8,4.9,2.0
71,3,7.7,2.8,6.7,2.0
72,3,6.3,2.7,4.9,1.8


In [5]:
gravity_centers = train_data.groupby(train_data.Class).mean()
gravity_centers

,P1,P2,P3,P4
Class,,,,
1,5.025,3.479167,1.4625,0.250
2,6.012,2.776000,4.3120,1.344
3,6.576,2.928000,5.6400,2.044


In [6]:
train_data_no_class = train_data.drop(columns=["Class"])
test_data_no_class = test_data.drop(columns=["Class"])

In [7]:
mv = train_data_no_class.mean()
mv

P1    5.882432
P2    3.055405
P3    3.836486
P4    1.225676
dtype: float64

In [12]:
sd = train_data_no_class.std()
sd

P1    0.857898
P2    0.468791
P3    1.805287
P4    0.765535
dtype: float64

In [13]:
test_data_no_class

,P1,P2,P3,P4
0,4.9,3.0,1.4,0.2
1,4.7,3.2,1.3,0.2
2,4.6,3.1,1.5,0.2
3,5.0,3.6,1.4,0.2
4,5.4,3.9,1.7,0.4
...,...,...,...,...
69,6.9,3.2,5.7,2.3
70,5.6,2.8,4.9,2.0
71,7.7,2.8,6.7,2.0
72,6.3,2.7,4.9,1.8


In [16]:
gravity_centers_std = (gravity_centers-mv)/sd
gravity_centers_std

,P1,P2,P3,P4
Class,,,,
1,-0.999458,0.903945,-1.315019,-1.274501
2,0.151029,-0.596013,0.263400,0.154564
3,0.808450,-0.271774,0.999018,1.068957


In [15]:
test_data_std = (test_data_no_class-mv)/sd
test_data_std

,P1,P2,P3,P4
0,-1.145163,-0.118188,-1.349639,-1.339815
1,-1.378291,0.308442,-1.405032,-1.339815
2,-1.494855,0.095127,-1.294246,-1.339815
3,-1.028599,1.161700,-1.349639,-1.339815
4,-0.562343,1.801645,-1.183461,-1.078560
...,...,...,...,...
69,1.186118,0.308442,1.032253,1.403364
70,-0.329215,-0.544817,0.589110,1.011481
71,2.118630,-0.544817,1.586182,1.011481
72,0.486734,-0.758132,0.589110,0.750226


In [18]:
distances = pd.DataFrame()
for index, gravity_center_std in enumerate(gravity_centers_std.values):
    data = []
    for train_data_value in test_data_std.values:
        data.append( 
            np.sum((gravity_center_std - train_data_value) ** 2)
        )
    distances[f"D{index+1}"] = data
distances

,D1,D2,D3
0,1.071450,6.743497,15.158566
1,0.510507,8.173695,16.700126
2,0.904303,7.846042,16.501075
3,0.072751,9.316145,16.747973
4,1.052634,9.871664,15.553182
...,...,...,...
69,17.812012,4.040081,0.592216
70,11.399563,1.073649,1.540161
71,25.464062,6.358130,2.139188
72,12.696494,0.599880,0.609659


In [ ]:
distances.dtypes

In [21]:
classes = []
mins = []
for train_data_value in test_data_std.values:
    dists_to_center = []
    for gravity_center_std in gravity_centers_std.values:
        dists_to_center.append( 
            np.linalg.norm(gravity_center_std-train_data_value) # squre of euclidean dist
        )
    classes.append(np.argmin(dists_to_center)+1) # argmin returns index of min element
    mins.append(dists_to_center)

test_data_std["Dist"] = mins
test_data_std["FoundClass"] = classes
test_data_std

,P1,P2,P3,P4,Dist,FoundClass
0,-1.145163,-0.118188,-1.349639,-1.339815,"[1.0351088823061887, 2.5968244833143808, 3.893...",1
1,-1.378291,0.308442,-1.405032,-1.339815,"[0.7144979418427325, 2.858967391380996, 4.0865...",1
2,-1.494855,0.095127,-1.294246,-1.339815,"[0.9509483145366688, 2.8010786817793547, 4.062...",1
3,-1.028599,1.161700,-1.349639,-1.339815,"[0.26972478633543595, 3.052236124289497, 4.092...",1
4,-0.562343,1.801645,-1.183461,-1.078560,"[1.0259797330115425, 3.1419204116855477, 3.943...",1
...,...,...,...,...,...,...
69,1.186118,0.308442,1.032253,1.403364,"[4.22042797509048, 2.0099952554442897, 0.76955...",3
70,-0.329215,-0.544817,0.589110,1.011481,"[3.376323915370405, 1.0361701426154604, 1.2410...",2
71,2.118630,-0.544817,1.586182,1.011481,"[5.046192852026093, 2.5215331838942507, 1.4625...",3
72,0.486734,-0.758132,0.589110,0.750226,"[3.5632140286584963, 0.7745191665585185, 0.780...",2


In [24]:
np.sum(test_data_std.FoundClass==test_data.Class)

65

In [25]:
74-65

9